In [ ]:
# check gpu
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-032605d3-7988-b867-546d-28fe85d15854)


In [ ]:
# get latest version of tfds
![ ! -f "pip_installed" ] && pip install tensorflow-datasets==4.4.0
!touch pip_installed

     |████████████████████████████████| 4.0 MB 9.3 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
# mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# S3 bucket
S3BUCKET='s3://semisupercv-fourthbrain-july/xray_orig.zip'
MODELDIR = 's3://semisupercv-fourthbrain-july/'

In [ ]:
# unzip data to root/tensorflow_datasets
!unzip -qq {S3BUCKET} -d /root/tensorflow_datasets

In [ ]:
# Dont need to do this in Sagemaker, already starting out in Github folder
# clone repository to contents/TeamSemiSuperCV
#!git clone https://github.com/TeamSemiSuperCV/semi-super.git /contents/TeamSemiSuperCV

In [ ]:
# test dating loading
import tensorflow_datasets as tfds
print(tfds.__version__)
# tfds.load('tfds')
tfds.load('xray_orig')

4.4.0


{'test': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train_10pc': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train_1pc': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train_2pc': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train_5pc': <PrefetchDataset shapes: {image: (None, None, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [ ]:
# change dir to TeamSemiSuperCV (repo code)
import os
#os.chdir('/contents/TeamSemiSuperCV')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
!cd .. #move up a directory to main folder

In [ ]:
class simclrCommand():
  def __init__(self, model_dir, model_folder, params={}):
    #default params
    self.params = {'train_mode':"pretrain", 
          "mode":"train_then_eval",
          "train_batch_size":64, 
          "train_epochs":100 ,
          "learning_rate":1.0, 
          'weight_decay':1e-4, 
          "temperature":0.5, 
          "dataset":"xray", 
          "image_size":244, 
          "eval_split":"test", 
          "resnet_depth":50, 
          "use_blur":"False", 
          "color_jitter_strength":"0.5", 
          'model_dir': model_dir+model_folder ,
          "use_tpu":"False"}
    # add in or change params
    if len(params) > 0:
      for k,v in params.items():
        self.params[k]=v
  def compile_command(self):
    simclr_command = ['python run.py']
    for k,v in self.params.items():
      simclr_command.append(f'--{k}={v}')
    return (" ").join(simclr_command)
  def run_command(self):
    !{self.compile_command()}


# Parameters: sk_ratio=0.0625, lr1, 

In [ ]:
model_folder = 'xray_orig_2x_skTrue_lr07_wd3_temp25'
params = {"train_batch_size":64, 
          "width_multiplier":1 , 
          "sk_ratio":0.0625, 
          'dataset':"xray_orig",
          "learning_rate":0.7, 
          'weight_decay':1e-3, 
          "temperature":0.25} 
slimsk2 = simclrCommand(model_dir, model_folder2, params2)
slimsk2.run_command()